## Etiquetado de tweets

In [11]:
import pandas as pd
import os
from dotenv import load_dotenv
import openai
import asyncio

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

In [12]:
system_prompt = """You are a data scientist labelling tweets. Given a tweet, classify it into one of the following categories:

Tesla Products
Customer Experience
Performance & Innovation
Financial News
Environmental Impact
Industry News
Charging Infrastructure

If the tweet does not clearly fit into any of these categories or is not relevant, classify it as "Not relevant".

Example:

Tweet:

Tesla's new Model S Plaid is the fastest production car ever made!

Category:

Tesla Products

Example:

Tweet:

I'm having a terrible time with Tesla customer service. They've been ignoring my emails for weeks.

Category:

Customer Experience

I want you to be very rigorous with the labelling. You have to respond with the following phrase: Category: [response]"""

In [13]:
df = pd.read_csv('../../data/clean/tesla_tweets_unlabelled.csv')
test_df = df.sample(50)
df.head()

,Link,Date,Text
0,https://twitter.com/Jessica1988kk/status/15131...,2022-04-10 19:44:00,Crypto news Bitcoin Whales flying motorbikes w...
1,https://twitter.com/MmeCallas/status/151317374...,2022-04-10 19:45:00,love MariaCallas I KNOW Y art HOLDonLINE music...
2,https://twitter.com/BotSecx/status/15131737626...,2022-04-10 19:45:00,love MariaCallas I KNOW Y art HOLDonLINE music...
3,https://twitter.com/ElTendies/status/151317393...,2022-04-10 19:45:00,Tesla A Trillion Dollar Company Worlds Largest...
4,https://twitter.com/LauraCory2013/status/15131...,2022-04-10 19:45:00,chargingstations area I dont Tesla Powerwall a...


In [14]:
"""df_labelled = pd.DataFrame(columns=['Link', 'Date', 'Text', 'Category'])
progress = 0

for index, row in test_df.iterrows():
    completion = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": row['Text']}
      ],
        max_tokens=10,
        temperature=0.9,
    )
    answer = completion.choices[0].message['content'].replace('Category: ', '')
    df_labelled.loc[len(df_labelled.index)] = [row['Link'], row['Date'], row['Text'], answer]
    progress += 1
    if progress % 10 == 0:
      print(f"Progress: {progress}/{len(test_df.index)}")"""



'df_labelled = pd.DataFrame(columns=[\'Link\', \'Date\', \'Text\', \'Category\'])\nprogress = 0\n\nfor index, row in test_df.iterrows():\n    completion = openai.ChatCompletion.create(\n      model="gpt-3.5-turbo",\n      messages=[\n        {"role": "system", "content": system_prompt},\n        {"role": "user", "content": row[\'Text\']}\n      ],\n        max_tokens=10,\n        temperature=0.9,\n    )\n    answer = completion.choices[0].message[\'content\'].replace(\'Category: \', \'\')\n    df_labelled.loc[len(df_labelled.index)] = [row[\'Link\'], row[\'Date\'], row[\'Text\'], answer]\n    progress += 1\n    if progress % 10 == 0:\n      print(f"Progress: {progress}/{len(test_df.index)}")'

In [15]:
def send_chatgpt_query(prompt : str):
    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt}
        ],
        max_tokens=10,
        temperature=0.9,
    )
    return completion.choices[0]


for index, row in test_df.iterrows():
    print(row['Text'])
    print(send_chatgpt_query(row['Text']).message['content'].replace('Category: ', ''))
    print('-----------------')



<OpenAIObject at 0x193605963f0> JSON: {
  "index": 0,
  "message": {
    "role": "assistant",
    "content": "Category: Customer Experience"
  },
  "finish_reason": "stop"
}

In [16]:
"""df_aux = pd.read_csv('../../data/clean/tesla_tweets_labelled.csv')

#comprobar que ninguna fila coincide con la que ya esta en el df_aux

mask = df_labelled['Link'].isin(df_aux['Link'])
df_labelled = df_labelled[~mask]

pd.concat([df_aux, df_labelled]).to_csv('../../data/clean/tesla_tweets_labelled.csv', index=False)"""

"df_aux = pd.read_csv('../../data/clean/tesla_tweets_labelled.csv')\n\n#comprobar que ninguna fila coincide con la que ya esta en el df_aux\n\nmask = df_labelled['Link'].isin(df_aux['Link'])\ndf_labelled = df_labelled[~mask]\n\npd.concat([df_aux, df_labelled]).to_csv('../../data/clean/tesla_tweets_labelled.csv', index=False)"

In [19]:
df = pd.read_csv('../../data/clean/tesla_tweets_labelled.csv')
print(df.shape)
df.values.tolist()

(995, 4)


[['https://twitter.com/Businesblog5/status/1579966415058403337',
  '2022-10-12 02:15:00',
  'I drove sports electric SUVs Tesla Kia TSLA',
  'Tesla Products'],
 ['https://twitter.com/mhannah74/status/1587228635420770305',
  '2022-11-01 03:12:00',
  'RenoNV half voters line never received ballots though system says mailed voters USPS compromised conspiring DNC fraud Tesla gigafactory',
  'Not relevant'],
 ['https://twitter.com/HopeVoteBiden/status/1528596954028158977',
  '2022-05-23 09:11:00',
  'Since US governments road safety agency sent teams crashes systems either use suspected operating Of involved Teslas according NHTSA document released Wednesday Tesla ElonMusk TeslaStock Twitter',
  'Industry News'],
 ['https://twitter.com/Zxcxz_xyz/status/1583606651722301441',
  '2022-10-22 03:20:00',
  'New Tesla Model Y Colors Quicksilver amp Midnight Cherry Red Tesla EV ClimateChange Renewables RenewableEnergy',
  'Tesla Products'],
 ['https://twitter.com/MANSOURSTOCKS_/status/1576870411534